In [1]:
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource

ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [2]:
edges = cg.read_chunk_edges([cg.get_chunk_id(layer=2,x=50,y=100,z=0)])
edges = edges["between"].get_pairs()
nodes = np.unique(edges)
parents = cg.get_roots(nodes)
d = dict(zip(parents, nodes))
print(d)

{864691138338302132: 75582971913961693, 864691137629078617: 75512534450307707, 864691137995403800: 75582903194484819, 864691137995404568: 75582834475009044, 864691138741942725: 75582971913961542, 864691137995404824: 75582903194484963, 864691136334203586: 75582971913961503}


In [3]:
edges = [[75582971913961693, 75582971913961503]]
print(cg.get_cross_chunk_edges_layer(edges))
print(cg.get_parents(edges[0]))

[1]
[147640565951889409 147640565951889410]


In [4]:
parents = np.array(edges[0][0])
while parents and parents.size:
    parents = cg.get_parent(parents)
    print(parents)

147640565951889409
219704963218014230
291635563662936192
363718446468361184
435828817064694903
506866064312035921
576742227280767262
864691138338302132
None


In [5]:
from pychunkedgraph.graph.edits import add_edges
from pychunkedgraph.graph.utils.context_managers import TimeIt

edges = np.array(edges, dtype=np.uint64)
with TimeIt("add_edge"):
    print(add_edges(cg, atomic_edges=edges))

2 [147640565951889443]
2 [147640565951889443]
3 [219704963218014264]
[219704963218014264] {219704963218014264: array([147640565951889443, 147640497232412673, 147710934696067076,
       147640497232412683, 147640565951889409, 147640565951889410,
       147640497232412684], dtype=uint64)} {147640497232412673: {2: array([[75582903194485753, 75582971913961501],
       [75582903194485544, 75582971913961501],
       [75582903194485544, 75582971913961488],
       [75582903194485720, 75582971913961693],
       [75582903194485720, 75582971913961473],
       [75582903194485767, 75582971913961693]], dtype=uint64), 3: array([[75582903194485200, 75512534450307435],
       [75582903194485200, 75512534450307545],
       [75582903194485330, 75512534450307653],
       [75582903194485481, 75512534450307737],
       [75582903194485490, 75512534450307910],
       [75582903194485528, 75512534450307957],
       [75582903194485720, 75512534450307957],
       [75582903194485720, 75512534450308044],
       [75

KeyError: 147640565951889443